In [27]:
from skimage.metrics import structural_similarity as ssim
import cv2
import numpy as np

# Load reference clear video and the dehazed video
reference_video = cv2.VideoCapture('test.mp4')
dehazed_video = cv2.VideoCapture('ded_output.avi')

# Check if videos were opened successfully
if not reference_video.isOpened() or not dehazed_video.isOpened():
    print("Error: Could not open video files.")
else:
    # Get total frame counts for both videos
    ref_frames = int(reference_video.get(cv2.CAP_PROP_FRAME_COUNT))
    dehaze_frames = int(dehazed_video.get(cv2.CAP_PROP_FRAME_COUNT))

    # Get the minimum number of frames between the two videos
    min_frames = min(ref_frames, dehaze_frames)

    # Initialize variables to accumulate SSIM values and count frames
    ssim_values = []
    frame_count = 0

    # Process video frames up to the length of the shortest video
    while frame_count < min_frames:
        ret_ref, frame_ref = reference_video.read()
        ret_dehaze, frame_dehaze = dehazed_video.read()

        # Check if frames were read successfully from both videos
        if not ret_ref or not ret_dehaze:
            print("Error: Could not read frames from one or both videos.")
            break

        # Remove potential singleton dimensions in frames
        frame_ref = np.squeeze(frame_ref)
        frame_dehaze = np.squeeze(frame_dehaze)

        # Ensure frames are larger than the default SSIM window size (7x7)
        frame_ref = cv2.resize(frame_ref, (100, 100))  # Resize frames as needed
        frame_dehaze = cv2.resize(frame_dehaze, (100, 100))  # Resize frames as needed

        # Calculate the maximum allowable window size based on the smaller side of the frames
        max_win_size = min(frame_ref.shape[0], frame_ref.shape[1])

        ssim_frame = ssim(frame_ref, frame_dehaze, win_size=min(3, max_win_size), multichannel=True)

        ssim_values.append(ssim_frame)

        # Increment frame count
        frame_count += 1

    # Check if frames were processed
    if frame_count == 0:
        print("No frames processed.")
    else:
        # Calculate average SSIM if frames were processed
        average_ssim = sum(ssim_values) / frame_count
        print(f"Average SSIM: {average_ssim}")

# Release video resources
reference_video.release()
dehazed_video.release()


Average SSIM: 0.24056862249771366


In [8]:
ref_frames

383

In [28]:
ssim_value = 0.24056862249771366
ssim_percentage = ((ssim_value + 1) / 2) * 100
print(f"SSIM Percentage: {ssim_percentage:.2f}%")


SSIM Percentage: 62.03%
